In [32]:
import pandas as pd
import requests

import datetime

## 1. Данные по ипотечным ставкам

Источник: ЦБ РФ
Период: 2014–2024 (10 лет)

- Ключевая ставка ЦБ РФ (%) - https://cbr.ru/hd_base/KeyRate/
- Средний уровень процентной ставки по ипотечному кредиту, региональный разрез (%) - https://www.fedstat.ru/indicator/59319
- Средневзвешенная процентная ставка по ипотечным жилищным кредитам в рублях, региональный разрез (%) - https://www.fedstat.ru/indicator/60293 


In [33]:
start_date =  datetime.date(2014, 1, 1).strftime('%d.%m.%Y')
end_date =  datetime.date(2024, 12, 31).strftime('%d.%m.%Y')

# Ключевая ставка 
url_key_rate = f'https://cbr.ru/hd_base/KeyRate/?UniDbQuery.Posted=True&UniDbQuery.From={start_date}&UniDbQuery.To={end_date}'

response = requests.get(url_key_rate)
response.raise_for_status()

key_rate_df = pd.read_html(url_key_rate)[0]

key_rate_df.columns = ['date', 'key_rate']
key_rate_df['date'] = pd.to_datetime(key_rate_df['date'], format='%d.%m.%Y')
key_rate_df['key_rate'] = key_rate_df['key_rate']/100

if(~key_rate_df.isna().any().any()):
    print('В таблице отсутствуют значения NaN\n')
else:
    print('В таблице есть значения NaN\n')
key_rate_df.info()
key_rate_df

В таблице отсутствуют значения NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2751 entries, 0 to 2750
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      2751 non-null   datetime64[ns]
 1   key_rate  2751 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 43.1 KB


,date,key_rate
0,2024-12-30,21.0
1,2024-12-28,21.0
2,2024-12-27,21.0
3,2024-12-26,21.0
4,2024-12-25,21.0
...,...,...
2746,2014-01-15,5.5
2747,2014-01-14,5.5
2748,2014-01-13,5.5
2749,2014-01-10,5.5


In [34]:
# Средний уровень процентной ставки по ипотечному кредиту, региональный разрез (%)

file_path = r'..\data\home_average_mortgage_rate_region.xls'
home_average_mortgage_rate_df = pd.read_excel(file_path, sheet_name='Данные', skiprows=2)
home_average_mortgage_rate_df.columns = ['region','year', 'month', 'avg_rate']

if(~home_average_mortgage_rate_df.isna().any().any()):
    print('В таблице отсутствуют значения NaN\n')
else:
    print('В таблице есть значения NaN\n')

home_average_mortgage_rate_df.info()
home_average_mortgage_rate_df


В таблице отсутствуют значения NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7492 entries, 0 to 7491
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   region    7492 non-null   object 
 1   year      7492 non-null   int64  
 2   month     7492 non-null   object 
 3   avg_rate  7492 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 234.3+ KB


,region,year,month,avg_rate
0,Российская Федерация,2018,значение показателя за год,9.56
1,Российская Федерация,2019,январь,9.88
2,Российская Федерация,2019,февраль,10.04
3,Российская Федерация,2019,март,10.18
4,Российская Федерация,2019,апрель,10.28
...,...,...,...,...
7487,Чукотский автономный округ,2024,август,8.38
7488,Чукотский автономный округ,2024,сентябрь,8.57
7489,Чукотский автономный округ,2024,октябрь,8.86
7490,Чукотский автономный округ,2024,ноябрь,8.85


In [35]:
# Средневзвешенная процентная ставка по ипотечным жилищным кредитам в рублях, региональный разрез (%)

file_path = r'..\data\home_weight_avg_mortgage_rate_region.xls'
home_weight_avg_mortgage_rate_df = pd.read_excel(file_path, sheet_name='Данные', skiprows=2)
home_weight_avg_mortgage_rate_df.columns = ['region','year', 'month', 'weight_avg_rate']

if(~home_weight_avg_mortgage_rate_df.isna().any().any()):
    print('В таблице отсутствуют значения NaN\n')
else:
    print('В таблице есть значения NaN\n')

home_weight_avg_mortgage_rate_df.info()
home_weight_avg_mortgage_rate_df

В таблице отсутствуют значения NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7492 entries, 0 to 7491
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   region           7492 non-null   object 
 1   year             7492 non-null   int64  
 2   month            7492 non-null   object 
 3   weight_avg_rate  7492 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 234.3+ KB


,region,year,month,weight_avg_rate
0,Российская Федерация,2018,значение показателя за год,9.56
1,Российская Федерация,2019,январь,9.88
2,Российская Федерация,2019,февраль,10.04
3,Российская Федерация,2019,март,10.18
4,Российская Федерация,2019,апрель,10.28
...,...,...,...,...
7487,Чукотский автономный округ,2024,август,8.38
7488,Чукотский автономный округ,2024,сентябрь,8.57
7489,Чукотский автономный округ,2024,октябрь,8.86
7490,Чукотский автономный округ,2024,ноябрь,8.85


In [36]:
# Ипотечные жилищные кредиты, предоставленные физическим лицам-резидентам,
# и приобретенные права требования по ипотечным жилищным кредитам

xlsx_file_path = r'..\data\02_02_Mortgage.xlsx'

mortgage_df = pd.read_excel(xlsx_file_path, sheet_name='в рублях', skiprows=1)
mortgage_df = mortgage_df.set_index('Отчетная дата').T
mortgage_df.columns = [
    'loans_issued_monthly_count',
    'loans_issued_monthly_volume_millions_rub',
    'total_debt_millions_rub',
    'overdue_debt_millions_rub',
    'acquired_mortgage_rights_debt_millions_rub',
    'total_debt_including_acquired_rights_millions_rub',
    'weighted_average_loan_term_months',
    'weighted_average_interest_rate_percent'
]

# Добавим новый признак 'Тип ипотеки' и разбиение 'новостройка = 1', 'вторичное жилье = 0'
mortgage_df['mortgage_type'] = 0

# Ипотечные жилищные кредиты, предоставленные физическим лицам-резидентам 
# под залог прав требования по договорам участия в долевом строительстве (ДДУ), 
# и приобретенные права требования по ипотечным жилищным кредитам под залог ДДУ

xlsx_file_path = r'..\data\02_03_Scpa_mortgage.xlsx'

scpa_mortgage_df = pd.read_excel(xlsx_file_path, sheet_name='в рублях', skiprows=[0, 10])
scpa_mortgage_df = scpa_mortgage_df.set_index('Отчетная дата').T
scpa_mortgage_df.columns = [
    'loans_issued_monthly_count',
    'loans_issued_monthly_volume_millions_rub',
    'total_debt_millions_rub',
    'overdue_debt_millions_rub',
    'acquired_mortgage_rights_debt_millions_rub',
    'total_debt_including_acquired_rights_millions_rub',
    'weighted_average_loan_term_months',
    'weighted_average_interest_rate_percent'
]

# Добавим новый признак 'Тип ипотеки' и разбиение 'новостройка = 1', 'вторичное жилье = 0'
scpa_mortgage_df['mortgage_type'] = 1

# TODO Совместим два датафрейма

home_info_mortgage_df = pd.concat([mortgage_df, scpa_mortgage_df], ignore_index=True)

if(~home_info_mortgage_df.isna().any().any()):
    print('В таблице отсутствуют значения NaN\n')
else:
    print('В таблице есть значения NaN\n')

home_info_mortgage_df.info()
home_info_mortgage_df
mortgage_df

В таблице отсутствуют значения NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 9 columns):
 #   Column                                             Non-Null Count  Dtype 
---  ------                                             --------------  ----- 
 0   loans_issued_monthly_count                         170 non-null    object
 1   loans_issued_monthly_volume_millions_rub           170 non-null    object
 2   total_debt_millions_rub                            170 non-null    object
 3   overdue_debt_millions_rub                          170 non-null    object
 4   acquired_mortgage_rights_debt_millions_rub         170 non-null    object
 5   total_debt_including_acquired_rights_millions_rub  170 non-null    object
 6   weighted_average_loan_term_months                  170 non-null    object
 7   weighted_average_interest_rate_percent             170 non-null    object
 8   mortgage_type                                      170 non-null  

,loans_issued_monthly_count,loans_issued_monthly_volume_millions_rub,total_debt_millions_rub,overdue_debt_millions_rub,acquired_mortgage_rights_debt_millions_rub,total_debt_including_acquired_rights_millions_rub,weighted_average_loan_term_months,weighted_average_interest_rate_percent,mortgage_type
01.01.2018,151308.0,290469.0,5144935.0,54575.0,134733.0,5279668.0,187.6,9.79,0
01.02.2018,78043.0,148275.0,5184710.0,55965.0,133665.0,5318375.0,191.1,9.85,0
01.03.2018,102654.0,198940.0,5272293.0,56636.0,138745.0,5411038.0,190.0,9.75,0
01.04.2018,118702.0,234757.0,5381205.0,56469.0,139759.0,5520964.0,193.4,9.63,0
01.05.2018,125300.0,249267.0,5500335.0,57186.0,138488.0,5638823.0,189.5,9.60,0
...,...,...,...,...,...,...,...,...,...
01.09.2024,100122.0,375068.0,19284915.0,76278.0,556224.0,19841139.0,298.3,9.32,0
01.10.2024,94348.0,372813.0,19227069.0,81075.0,572505.0,19799575.0,302.8,8.94,0
01.11.2024,94934.0,367052.0,19106419.0,86887.0,834976.0,19941395.0,299.3,9.65,0
01.12.2024,72315.0,274353.0,19118351.0,91008.0,903532.0,20021884.0,302.2,9.01,0


### Описание таблицы ипотечной ставки 'mortgage_df' и 'scpa_mortgage_df'

__ИЖК (Ипотечное жилищное кредитование)__  

__ИЖК по ДДУ (Ипотечное жилищное кредитование по договорам долевого участия)__   

| Имя столбца в таблице                                      | Описание                                                                 |
|------------------------------------------------------------|--------------------------------------------------------------------------|
| `loans_issued_monthly_count`                               | Количество предоставленных кредитов за месяц, единиц                    |
| `loans_issued_monthly_volume_millions_rub`                 | Объем предоставленных кредитов за месяц, млн руб.                       |
| `total_debt_millions_rub`                                  | Задолженность по предоставленным кредитам, млн руб., в том числе        |
| `overdue_debt_millions_rub`                                | Просроченная задолженность по предоставленным кредитам, млн руб.        |
| `acquired_mortgage_rights_debt_millions_rub`               | Задолженность по приобретенным правам требования по ипотечным кредитам, млн руб. |
| `total_debt_including_acquired_rights_millions_rub`        | Задолженность по предоставленным кредитам с учетом приобретенных прав требования, млн руб. |
| `weighted_average_loan_term_months`                        | Средневзвешенный срок кредитования по кредитам, выданным в течение месяца, месяцев |
| `weighted_average_interest_rate_percent`                   | Средневзвешенная ставка по кредитам, выданным в течение месяца, %       |


## 2. Данные по ценам на недвижимость

Источник: Росстат, Kaggle, порталы недвижимости (ЦИАН, Avito), Домклик   
Период: 2014-2024

- Средняя цена за квадратный метр (руб) -  https://www.fedstat.ru/indicator/31452
- Индекс цен на рынке жилья по регионам , региональный разрез (%) - https://www.fedstat.ru/indicator/30925


In [37]:
# Средняя цена 1 кв. м общей площади квартир на рынке жилья  (рубль)

file_path = r'..\data\home_average_price_region.xls'
home_average_price_df = pd.read_excel(file_path, sheet_name='Данные', skiprows=2)
home_average_price_df.columns = ['type_of_housing_market','region','year', 'quarter', 'price']

if(~home_average_price_df.isna().any().any()):
    print('В таблице отсутствуют значения NaN\n')
else:
    print('В таблице есть значения NaN\n')

home_average_price_df.info()
home_average_price_df

В таблице отсутствуют значения NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8034 entries, 0 to 8033
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   type_of_housing_market  8034 non-null   object 
 1   region                  8034 non-null   object 
 2   year                    8034 non-null   int64  
 3   quarter                 8034 non-null   object 
 4   price                   8034 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 314.0+ KB


,type_of_housing_market,region,year,quarter,price
0,Вторичный рынок жилья,Российская Федерация,2014,I квартал,55924.70
1,Вторичный рынок жилья,Российская Федерация,2014,II квартал,56630.42
2,Вторичный рынок жилья,Российская Федерация,2014,III квартал,57119.36
3,Вторичный рынок жилья,Российская Федерация,2014,IV квартал,58084.71
4,Вторичный рынок жилья,Российская Федерация,2015,I квартал,58707.41
...,...,...,...,...,...
8029,Первичный рынок жилья,Чукотский автономный округ,2024,III квартал,233799.46
8030,Первичный рынок жилья,Крымский федеральный округ,2016,I квартал,51250.42
8031,Первичный рынок жилья,Крымский федеральный округ,2016,II квартал,50877.34
8032,Первичный рынок жилья,Крымский федеральный округ,2016,III квартал,51239.18


In [38]:
# Индексы цен на рынке жилья (%)

file_path = r'..\data\home_index_price_region.xls'
home_index_price_df = pd.read_excel(file_path, sheet_name='Данные', skiprows=3)
home_index_price_df.columns = ['type_of_housing_market', 'region', 'year', 'quarter', 'index']

if(~home_index_price_df.isna().any().any()):
    print('В таблице отсутствуют значения NaN\n')
else:
    print('В таблице есть значения NaN\n')

home_index_price_df.info()
home_index_price_df[home_index_price_df['year'] == 2024]

В таблице отсутствуют значения NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7844 entries, 0 to 7843
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   type_of_housing_market  7844 non-null   object 
 1   region                  7844 non-null   object 
 2   year                    7844 non-null   int64  
 3   quarter                 7844 non-null   object 
 4   index                   7844 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 306.5+ KB


,type_of_housing_market,region,year,quarter,index
40,Вторичный рынок жилья,Российская Федерация,2024,I квартал,108.00
81,Вторичный рынок жилья,Центральный федеральный округ,2024,I квартал,107.56
82,Вторичный рынок жилья,Центральный федеральный округ,2024,III квартал,104.48
123,Вторичный рынок жилья,Белгородская область,2024,I квартал,102.87
124,Вторичный рынок жилья,Белгородская область,2024,III квартал,104.89
...,...,...,...,...,...
7803,Первичный рынок жилья,Сахалинская область,2024,III квартал,103.86
7836,Первичный рынок жилья,Еврейская автономная область,2024,I квартал,130.95
7837,Первичный рынок жилья,Еврейская автономная область,2024,III квартал,118.74
7842,Первичный рынок жилья,Чукотский автономный округ,2024,I квартал,112.77


## 3. Макроэкономические данные

Источник: Росстат, ЦБ РФ  
Период: 2014-2024

- Уровень инфляции (%) - https://cbr.ru/statistics/ddkp/infl/?utm_source=w&utm_content=page
- Среднедушевые денежные доходы населения, региональный разрез (руб)
- Курс доллара (руб)


In [39]:
start_date =  datetime.date(2014, 1, 1).strftime('%d.%m.%Y')
end_date =  datetime.date(2024, 12, 31).strftime('%d.%m.%Y')

# Ключевая ставка + Инфляция

url = f'https://cbr.ru/statistics/ddkp/infl/?UniDbQuery.Posted=True&UniDbQuery.From={start_date}&UniDbQuery.To={end_date}'

response = requests.get(url)
response.raise_for_status()

key_rate_inflation_df = pd.read_html(url)[0]

key_rate_inflation_df.columns = ['date', 'key_rate', 'inflation', 'target_inflation']
key_rate_inflation_df['key_rate'] = key_rate_inflation_df['key_rate']/100
key_rate_inflation_df['inflation'] = key_rate_inflation_df['inflation']/100
key_rate_inflation_df = key_rate_inflation_df.drop('target_inflation', axis=1)

if(~key_rate_inflation_df.isna().any().any()):
    print('В таблице отсутствуют значения NaN\n')
else:
    print('В таблице есть значения NaN\n')
key_rate_inflation_df.info()
key_rate_inflation_df

В таблице отсутствуют значения NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       132 non-null    float64
 1   key_rate   132 non-null    float64
 2   inflation  132 non-null    float64
dtypes: float64(3)
memory usage: 3.2 KB


,date,key_rate,inflation
0,12.2024,21.0,9.52
1,11.2024,21.0,8.88
2,10.2024,21.0,8.54
3,9.2024,19.0,8.63
4,8.2024,18.0,9.05
...,...,...,...
127,5.2014,7.5,7.59
128,4.2014,7.5,7.33
129,3.2014,7.0,6.92
130,2.2014,5.5,6.21


In [40]:
# Среднедушевые денежные доходы населения

file_path = r'..\data\avg_monetary_income_people_region.xlsx'

avg_monetary_income_people_df = pd.read_excel(file_path, skiprows=2)
avg_monetary_income_people_df = avg_monetary_income_people_df.drop([97, 98, 99, 100, 101])

# Таблица представлена не осень удобном формате, поэтому обработаем ее
collection_df_list = []

for i in range(1, len(avg_monetary_income_people_df)):   
    selected_rows = avg_monetary_income_people_df.iloc[[0, i], :]
    selected_rows = selected_rows.transpose()
    region = selected_rows.iloc[0, 1]
    selected_rows = selected_rows.drop('Unnamed: 0')
    selected_rows = selected_rows.reset_index()
    selected_rows['region'] = region
    selected_rows.columns = ['year', 'quarter', 'income', 'region']
    collection_df_list.append(selected_rows)

avg_monetary_income_people_df = pd.concat(collection_df_list, axis=0, ignore_index=True)
avg_monetary_income_people_df['year'] = avg_monetary_income_people_df['year'].replace(r'Unnamed:*', pd.NA, regex=True)
avg_monetary_income_people_df['year'] = avg_monetary_income_people_df['year'].ffill()

avg_monetary_income_people_df.info()
avg_monetary_income_people_df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6144 entries, 0 to 6143
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   year     6144 non-null   object
 1   quarter  6048 non-null   object
 2   income   6112 non-null   object
 3   region   6144 non-null   object
dtypes: object(4)
memory usage: 192.1+ KB


,year,quarter,income,region
0,2011\nгод,NaN,20771.642488,Российская Федерация
1,2012 год,I кв.,19105.67146,Российская Федерация
2,2012 год,II кв.,22572.96167,Российская Федерация
3,2012 год,III кв.,23262.024407,Российская Федерация
4,2012 год,IV кв.,27963.833212,Российская Федерация
...,...,...,...,...
6139,2023 год,IV кв. 2),156092,Чукотский авт.округ
6140,2023 год,год 3),156988,Чукотский авт.округ
6141,2024 год 2),I кв.,147290,Чукотский авт.округ
6142,2024 год 2),II кв.,157555,Чукотский авт.округ


In [41]:
# Курс доллара
start_date =  datetime.date(2014, 1, 1).strftime('%d/%m/%Y')
end_date =  datetime.date(2024, 12, 31).strftime('%d/%m/%Y')

xml_url = f'http://www.cbr.ru/scripts/XML_dynamic.asp?date_req1={start_date}&date_req2={end_date}&VAL_NM_RQ=R01235'

us_rate_df = pd.read_xml(xml_url)

if(~us_rate_df.isna().any().any()):
    print('В таблице отсутствуют значения NaN\n')
else:
    print('В таблице есть значения NaN\n')

us_rate_df.info()
us_rate_df

В таблице отсутствуют значения NaN

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2715 entries, 0 to 2714
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date       2715 non-null   object
 1   Id         2715 non-null   object
 2   Nominal    2715 non-null   int64 
 3   Value      2715 non-null   object
 4   VunitRate  2715 non-null   object
dtypes: int64(1), object(4)
memory usage: 106.2+ KB


,Date,Id,Nominal,Value,VunitRate
0,01.01.2014,R01235,1,"32,6587","32,6587"
1,10.01.2014,R01235,1,"33,1547","33,1547"
2,11.01.2014,R01235,1,"33,2062","33,2062"
3,14.01.2014,R01235,1,"33,1204","33,1204"
4,15.01.2014,R01235,1,"33,2386","33,2386"
...,...,...,...,...,...
2710,25.12.2024,R01235,1,"99,8729","99,8729"
2711,26.12.2024,R01235,1,"99,6125","99,6125"
2712,27.12.2024,R01235,1,"99,2295","99,2295"
2713,28.12.2024,R01235,1,"100,5281","100,5281"


In [42]:
# TODO: спарсить ипотечную ставку с сайта о недвижимости по типу ДомКлик
# так как с сайта цб подозрительно маленькая ставка